## Get FarmOS token

In [ ]:
import json
from pyodide import to_js
from IPython.display import JSON
from js import Object, fetch

token_url="https://waterbearcollective.farmos.net/oauth/token"

username="XXXX"
password="XXXX"
full_body = "grant_type=password&username="+username+"&password="+password+"&client_id=farm&scope=farm_manager"
resp = await fetch(token_url,
  method="POST",
body=full_body,  
  headers=Object.fromEntries(to_js({ "Content-Type": "application/x-www-form-urlencoded" })),
)
res = await resp.text()
rj=json.loads(res)
token=rj['access_token']
token_string='Bearer '+rj['access_token']
print(token)

## Retrieve land assets

In [ ]:
url="https://waterbearcollective.farmos.net/api/asset/land"
resp = await fetch(url,
  method="GET",
  headers=Object.fromEntries(to_js({ "Authorization": token_string })),
)
res = await resp.text()
JSON(json.loads(res))


## Create a land asset

In [ ]:
url="https://waterbearcollective.farmos.net/api/asset/land"

json_to_send = {
  "data": {
    "type": "asset--land",
    "attributes": {
     "name": "Wheeee",
     "land_type":"bed",
     "timestamp": "1526584271"
    }
  }
}

resp = await fetch(url,
  method="POST",
   body=json.dumps(json_to_send),
    headers=Object.fromEntries(to_js({ "Authorization": token_string,"Content-Type": "application/vnd.api+json"}))
)
res = await resp.text()
JSON(json.loads(res))



## Retrieve the API schema

In [ ]:
url="https://waterbearcollective.farmos.net/api/schema"

resp = await fetch(url,
  method="GET",
  #body = "grant_type=password&username=donblair&password=farmoscat999&client_id=farm&scope=farm_manager",
  #body=json.dumps({ "some" : "json" }),
  #credentials="same-origin",
                   
  headers=Object.fromEntries(to_js({ "Authorization": token_string })),
  #headers=Object.fromEntries(to_js({ "Content-Type": "application/json" })),
)
res = await resp.text()
JSON(json.loads(res))


## Import data from CSV file

In [ ]:
import pandas as pd

In [41]:
# https://github.com/jupyterlite/jupyterlite/issues/119

import micropip
package_url = "https://raw.githubusercontent.com/innovationOUtside/ouseful_jupyterlite_utils/main/ouseful_jupyterlite_utils-0.0.1-py3-none-any.whl"
await micropip.install(package_url)

import micropip

package_url = "https://raw.githubusercontent.com/innovationOUtside/ouseful_jupyterlite_utils/main/ouseful_jupyterlite_utils-0.0.1-py3-none-any.whl"

await micropip.install(package_url)

from ouseful_jupyterlite_utils import pandas_utils as pdu
# if CSV data is at a remote URL, use this approach
#from ouseful_jupyterlite_utils import pandas_utils as pdu# Load CSV from URL
# Via @jtpio
#URL = "https://support.staffbase.com/hc/en-us/article_attachments/360009197031/username.csv"
#df = await pdu.read_csv_url(URL, "\t")
#df

# Load CSV from local browser storage
#Via @bollwyvl
df = await pdu.read_csv_local("my_farm_data.csv", ",")
df


,name,type
0,Green Pasture,paddock
1,Pigeon Hill,landmark
2,Mule's Lament,landmark
3,Red Valley,landmark


In [ ]:
url="https://waterbearcollective.farmos.net/api/asset/land"

for i in range(0,len(df)):
    
    land_name = df['name'][i]
    land_type = df['type'][i]
    
    jdict = {
      "data": {
        "type": "asset--land",
        "attributes": {
         "name": land_name,
         "land_type":land_type,
         "timestamp": "1526584271"
        }
      }
    }

    print(jdict)

    resp = await fetch(url,
    method="POST",
    body=json.dumps(jdict),
    headers=Object.fromEntries(to_js({ "Authorization": token_string,"Content-Type": "application/vnd.api+json"}))
    )
    res = await resp.text()
    JSON(json.loads(res))
